In [104]:
import pandas as pd
from pandas import Series
import os
import numpy as np


import warnings
warnings.filterwarnings('ignore')

### Load Data

In [105]:
cwd = os.getcwd()
material_price_file = cwd + os.sep + "data" + os.sep + "Material Requisition.xlsx"
suppliers_file = cwd + os.sep + "data" + os.sep + "Suppliers.xlsx"
write_results_file = cwd + os.sep + "data" + os.sep + "MaterialResults.csv"

In [106]:
df_materials = pd.read_excel(material_price_file)
df_suppliers = pd.read_excel(suppliers_file)

In [107]:
df_features = df_materials

In [94]:
# BAD DATA POINT LOADING IN DATE FOR ORDER
df_features = df_features[df_features["MREQ_Number"].str.contains("MREQ-17251-001025")==False]

In [108]:
df_features[df_features["MREQ_Number"].str.contains("MREQ-17251-001025")==True]

,MREQ_Number,FFID,Date_and_Time_of_Request,Section,Order_Type,Item_Category,Item_Description,Quantity_Required,Unit,Unit_Price,Total_Cost,Category,Date_Required,Delivery_Location,Location_WBS,Activity_WBS,Current_Stage,Operational_Approval,Commercial_Approval,Buyer
4499,MREQ-17251-001025,17251,2019-05-13 09:45:32,Section 5,Material,Not on List,Not on List,3,2019-09-30 00:00:00,45,135,Surveying Consumables,2019-05-10 00:00:00,"Main Compound, Gate 1 - 1344810 - Cambridge Rd...",SEC5 - Roadworks,602 - Excavation (excl topsoil),Stores - Closed,Yes - Approve,Yes - Approve,Req - S5 Buyers


### CLEANING AND REMOVING BAD DATA - CASTING TO PROPER TYPES 

In [95]:
# CLEANING REMOVE ROWS
df_features1 = df_features[df_features["Category"].notna()]
df_features1 = df_features1[df_features1["Unit_Price"].notna()]
df_features1 = df_features1[df_features1["Item_Description"].notna()]
df_features1 = df_features1[df_features1["Unit"].notna()]

df_features1 = df_features1[df_features1["MREQ_Number"].str.contains("MREQ")==True]



In [96]:
# CAST VALUES
def is_number(x):
    try:
        float(x)
        return True
    except Exception as e:
        return False



df_features2 = df_features1[df_materials['Unit_Price'].map(is_number) == True]
df_features2["Unit_Price"] = pd.to_numeric(df_features2["Unit_Price"], downcast="float")
df_features2["Total_Cost"] = pd.to_numeric(df_features2["Total_Cost"], downcast="float")
df_features2["Unit"] = df_features2["Unit"].apply(str)


df_features2["Date_and_Time_of_Request"] = pd.to_datetime(df_features2["Date_and_Time_of_Request"])
df_features2["Date_and_Time_of_Request"] = df_features2["Date_and_Time_of_Request"].dt.date

df_features2["Date_Required"] = pd.to_datetime(df_features2["Date_Required"])
df_features2["Date_Required"] = df_features2["Date_Required"].dt.date


In [97]:
# Unit Type Cleaning
df_features2["Unit"] = df_features1["Unit"].replace("ea", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("EA", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("each", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("Each	", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("EACH", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("Ea", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("1", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("No", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("No.", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("no.", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("no", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("number", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("Number", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("N0", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("EAch", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("NO", "Each")


df_features2["Unit"] = df_features2["Unit"].replace("Units", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("Unit", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("unit", "Each")
df_features2["Unit"] = df_features2["Unit"].replace("units", "Each")


df_features2["Unit"] = df_features2["Unit"].replace("Rolls", "Roll")
df_features2["Unit"] = df_features2["Unit"].replace("roll", "Roll")
df_features2["Unit"] = df_features2["Unit"].replace("rolls", "Roll")

df_features2["Unit"] = df_features2["Unit"].replace("M", "Meters")
df_features2["Unit"] = df_features2["Unit"].replace("m", "Meters")
df_features2["Unit"] = df_features2["Unit"].replace("Meters", "Meters")
df_features2["Unit"] = df_features2["Unit"].replace("meters", "Meters")
df_features2["Unit"] = df_features2["Unit"].replace("Meter", "Meters")
df_features2["Unit"] = df_features2["Unit"].replace("Meter", "Metres")
df_features2["Unit"] = df_features2["Unit"].replace("metres", "Metres")


df_features2["Unit"] = df_features2["Unit"].replace("m2", "Meters2")
df_features2["Unit"] = df_features2["Unit"].replace("SM", "Meters2")

df_features2["Unit"] = df_features2["Unit"].replace("m3", "Meters3")
df_features2["Unit"] = df_features2["Unit"].replace("M3", "Meters3")

df_features2["Unit"] = df_features2["Unit"].replace("tonnes", "Tonnes")

df_features2["Unit"] = df_features2["Unit"].replace("bags", "Bags")
df_features2["Unit"] = df_features2["Unit"].replace("packs", "Packs")

df_features2["Unit"] = df_features2["Unit"].replace("tonne", "Ton")

df_features2["Unit"] = df_features2["Unit"].replace("box", "Box")
df_features2["Unit"] = df_features2["Unit"].replace("boxes", "Box")
df_features2["Unit"] = df_features2["Unit"].replace("Boxes", "Box")


df_features2["Unit"] = df_features2["Unit"].replace("tonne", "Ton")
df_features2["Unit"] = df_features2["Unit"].replace("ton", "Ton")

In [98]:
df_features3 = df_features2.groupby(["Category", "Item_Description","Unit"]).filter(lambda x : len(x)>2)
df_features3 = df_features3.groupby("Unit").filter(lambda x : len(x)>4)

### Feature Engineering

In [99]:
# Days to Delivery
df_features3["Days_To_Delivery"] = df_features3["Date_Required"] - df_features3["Date_and_Time_of_Request"]
df_features3["Days_To_Delivery"] = df_features3["Days_To_Delivery"].dt.days
df_features3[df_features3["Days_To_Delivery"]==pd.NaT] = None

In [115]:
# Outside Percentile Range

def find_anomalies(s:pd.Series,sd=2, groupby_cols=None):
    #define a list to accumlate anomalies
    
    
    # Set upper and lower limit to 3 standard deviation
    data_std = s
    data_mean = mean(s)
    anomaly_cut_off = data_std * sd
    
    lower_limit  = random_data_mean - anomaly_cut_off 
    upper_limit = random_data_mean + anomaly_cut_off
    print(lower_limit)
    # Generate outliers
    for outlier in random_data:
        if outlier > upper_limit or outlier < lower_limit:
            anomalies.append(outlier)
    return anomalies



df_features3["Price_Unit_Lower_Bound_2SD"] = df_features3[["Category", "Item_Description","Unit","Unit_Price"]]\
    .groupby(["Category", "Item_Description", "Unit"]).transform(lambda x : x.mean() - x.std()*1.5)

df_features3["Price_Unit_Lower_Bound_2SD"] = np.where(df_features3["Price_Unit_Lower_Bound_2SD"]<0,0,df_features3["Price_Unit_Lower_Bound_2SD"])

df_features3["Price_Unit_Upper_Bound_2SD"] = df_features3[["Category", "Item_Description","Unit","Unit_Price"]]\
    .groupby(["Category", "Item_Description", "Unit"]).transform(lambda x : x.mean() + x.std()*1.5)

df_features3["Unit_Price_Annomaly"] = np.where((df_features3["Unit_Price"]>=df_features3["Price_Unit_Lower_Bound_2SD"]) \
                                               & (df_features3["Unit_Price"]<=df_features3["Price_Unit_Upper_Bound_2SD"]),0, 1)

In [116]:
 df_features3[["Category", "Item_Description","Unit","Unit_Price"]]\
    .groupby(["Category", "Item_Description", "Unit"]).count()

Unit_Price
Category          Item_Description                       Unit               
Aggregate         6F5                                    Tonnes            7
                  6G                                     Tonnes           18
                  A10 Pipe Bed                           Tonnes            4
                  A20 Pipe Bed                           Tonnes           22
                  Bedding Sand                           Tonnes            4
...                                                                      ...
Waste (General)   Not on List                            Each             19
                                                         Meters3           4
Waste (Recycling) Metal - Roll On Roll Off 20 cubic yard Each              6
                  Metal - Roll On Roll Off 40 cubic yard Each              9
                  Not on List                            Each              4

[224 rows x 1 columns]

In [118]:
df_features3[df_features3["Unit_Price_Annomaly"]==True]

,MREQ_Number,FFID,Date_and_Time_of_Request,Section,Order_Type,Item_Category,Item_Description,Quantity_Required,Unit,Unit_Price,...,Location_WBS,Activity_WBS,Current_Stage,Operational_Approval,Commercial_Approval,Buyer,Days_To_Delivery,Price_Unit_Lower_Bound_2SD,Price_Unit_Upper_Bound_2SD,Unit_Price_Annomaly
14,MREQ-21746-001025,21746,2019-08-02,Section 3,Material,Concrete Drainage,CPM Chamber Ring 1200 x 250 D/S (913115),2,Each,55.410000,...,SC3B - Roadworks,502 - Chambers & Gullies,Buyer - Closed,Yes - Approve,Yes - Approve,REQ - Section 3 Buyers,3.0,29.334506,52.549489,1
20,MREQ-20693-001025,20693,2019-07-12,Core,Material,Not on List,Not on List,1,Each,115668.703125,...,PRLM - Preliminaries,104 - Site Transport,Buyer - Closed,Yes - Approve,Yes - Approve,REQ - Buyers,0.0,0.000000,19779.141357,1
92,MREQ-5613-00416,5613,2018-08-24,Section 4,Material,Not on List,Not on List,1,Each,325.500000,...,CL4B - Structures (Culverts) 4B,1802 - Miscellaneous Metalwork,Buyer - Closed,Yes - Approve,Yes - Approve,REQ - Section 4 Buyers,14.0,0.000000,209.150368,1
105,MREQ-21937-001025,21937,2019-08-07,Section 5,Material,Geotextile,Terram 1000 4.5m x 100m 1-15 rolls (820795),5,Each,122.720001,...,PLM5 - Site Preliminaries,153a - Temporary Works - Site Infrastructure,Buyer - Closed,Yes - Approve,Yes - Approve,Req - S5 Buyers,2.0,112.334076,120.400360,1
132,MREQ-14554-001025,14554,2019-03-22,Section 5,Material,"PPE, Small Tools and Consumables",Tape Measuring Pocket Spartan Pro 10mtr/3 (765...,5,Each,12.400000,...,PLM5 - Site Preliminaries,102 - Compound running costs,Operations Rejected,No - Reject,NaN,Req - S5 Buyers,5.0,12.399999,12.399999,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13429,MREQ-32686,32686,2020-08-05,Core,Material Requisition,Not on List,Not on List,1,Each,27000.000000,...,FEEC - Preliminaries,107 - Fee based costs including insurance,Closed - Buyers,Yes - Approve,Yes - Approve,REQ - Buyers,2.0,0.000000,18912.627441,1
13688,MREQ-33012,33012,2020-09-08,Section 4,Material Requisition,Not on List,Not on List,1,Each,4456.000000,...,PLM4 - Site Preliminaries,153b - Temporary Works - Waste Removal,Closed - Buyers,Yes - Approve,Yes - Approve,REQ - Section 4 Buyers,0.0,0.000000,3470.144897,1
13754,MREQ-33125,33125,2020-09-22,Section 4,Material Requisition,Kerbs,Concrete Kerbs Splayed 45deg 125x255 (898227),16,Each,6.340000,...,SC4A - Roadworks - Swavesey to Robins Lane,"1101 - Kerbs, Channels and Edgings",Closed - Buyers,Yes - Approve,Yes - Approve,REQ - Section 4 Buyers,0.0,3.882608,5.825965,1
13767,MREQ-33178,33178,2020-09-28,Section 6,Material Requisition,Aggregate,Not on List,300,Meters3,77.330002,...,SC6R - Roadworks Section 6,"1101 - Kerbs, Channels and Edgings",Closed - Buyers,Yes - Approve,Yes - Approve,Req - S6 Buyers,2.0,28.929789,71.722944,1


In [119]:
df_features3.to_csv(write_results_file)

In [103]:
df_features3

,MREQ_Number,FFID,Date_and_Time_of_Request,Section,Order_Type,Item_Category,Item_Description,Quantity_Required,Unit,Unit_Price,...,Location_WBS,Activity_WBS,Current_Stage,Operational_Approval,Commercial_Approval,Buyer,Days_To_Delivery,Price_Unit_Lower_Bound_2SD,Price_Unit_Upper_Bound_2SD,Unit_Price_Annomaly
3,MREQ-5559-00362,5559,2018-08-23,Section 4,Material,Not on List,Not on List,30,Each,17.450001,...,SC4A - Roadworks - Swavesey to Robins Lane,501 - Drains,Buyer - Closed,Yes - Approve,Yes - Approve,REQ - Section 4 Buyers,7.0,0.0,498.966457,0
4,MREQ-21737-001025,21737,2019-08-02,Core,Material,Not on List,Not on List,1,Each,20000.000000,...,WAOC - Working Areas overhead Costs,110 - Welfare Costs,Stores - Closed,Yes - Approve,Yes - Approve,REQ - Buyers,0.0,0.0,45117.829102,0
5,MREQ-21712-001025,21712,2019-08-02,Section 4,Material,Not on List,Not on List,4,Roll,258.049988,...,SC4B - Roadworks - Robins Lane to Dry Drayton,504 - Drainage and Service Ducts in Structures,Buyer - Closed,Yes - Approve,Yes - Approve,REQ - Section 4 Buyers,3.0,0.0,2455.215363,0
7,MREQ-21748-001025,21748,2019-08-02,Core,Material,Not on List,Not on List,1,Each,8750.000000,...,WAOC - Working Areas overhead Costs,123 - Venue / Meeting Costs (Hotel Conf etc),Stores - Closed,Yes - Approve,Yes - Approve,REQ - Buyers,3.0,0.0,11319.138184,0
8,MREQ-21748-001025,21748,2019-08-02,Core,Material,Not on List,Not on List,1,Each,473.399994,...,WAOC - Working Areas overhead Costs,123 - Venue / Meeting Costs (Hotel Conf etc),Stores - Closed,Yes - Approve,Yes - Approve,REQ - Buyers,3.0,0.0,11319.138184,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13895,MREQ-33336,33336,2020-10-19,Section 4,Material Requisition,Not on List,Not on List,40,Each,12.210000,...,PLM4 - Site Preliminaries,153d - Temporary Works - Scaffolding & Falsework,Buyers Queue,Yes - Approve,Yes - Approve,REQ - Section 4 Buyers,11.0,0.0,25638.811279,0
13896,MREQ-33336,33336,2020-10-19,Section 4,Material Requisition,Not on List,Not on List,40,Each,28.809999,...,PLM4 - Site Preliminaries,153d - Temporary Works - Scaffolding & Falsework,Buyers Queue,Yes - Approve,Yes - Approve,REQ - Section 4 Buyers,11.0,0.0,25638.811279,0
13897,MREQ-33336,33336,2020-10-19,Section 4,Material Requisition,Not on List,Not on List,38,Each,143.869995,...,PLM4 - Site Preliminaries,153d - Temporary Works - Scaffolding & Falsework,Buyers Queue,Yes - Approve,Yes - Approve,REQ - Section 4 Buyers,11.0,0.0,25638.811279,0
13900,MREQ-33339,33339,2020-10-19,Section 3,Material Requisition,Not on List,Not on List,1,Each,278.410004,...,PLM3 - Site Preliminaries,152 - Attendant / General Plant,Buyers Queue,Yes - Approve,Yes - Approve,REQ - Section 3 Buyers,11.0,0.0,25638.811279,0
